In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hw1_1 import *
from torch.optim import Adam
from torch.autograd import Variable

In [3]:
model = MyNetwork()

train_loader = loadData('hw1_data/hw1_data/p1_data/train_50',data_aug=True)
val_loader = loadData('hw1_data/hw1_data/p1_data/val_50')



loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.003)

In [4]:
def saveModel(name=''):
    path = "./hw1_1_myNet"+name+".pth"
    torch.save(model.state_dict(), path)

def valAccuracy():
    model.eval().cuda()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            outputs = model(images.cuda())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels.cuda()).sum().item()
    
    accuracy = (100 * accuracy / total)
    return(accuracy)

def train(num_epochs):
    
    best_accuracy = 0.0

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    model.to(device)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        for i, (images, labels) in enumerate(train_loader, 0):
            
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

        # Compute and print the average accuracy fo this epoch when tested over all test images
        accuracy = valAccuracy()
        print('For epoch', epoch+1,': test accuracy:{:.4f}%, loss:{:.4f}'.format(accuracy,loss.item()))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            #saveModel()
            best_accuracy = accuracy
        if epoch==1:
            saveModel('First')
        if epoch==18:
            saveModel('Mid')
            break

In [5]:
train(200)

The model will be running on cuda:0 device
For epoch 1 : test accuracy:3.5200%, loss:3.7987
For epoch 2 : test accuracy:6.8400%, loss:3.4259
For epoch 3 : test accuracy:7.7200%, loss:3.3945
For epoch 4 : test accuracy:8.3600%, loss:3.3506
For epoch 5 : test accuracy:9.8400%, loss:3.3559
For epoch 6 : test accuracy:12.5200%, loss:3.2262
For epoch 7 : test accuracy:14.2800%, loss:2.9376
For epoch 8 : test accuracy:16.4800%, loss:3.2594
For epoch 9 : test accuracy:16.8000%, loss:2.8969
For epoch 10 : test accuracy:21.6000%, loss:2.5370
For epoch 11 : test accuracy:25.8800%, loss:2.4910
For epoch 12 : test accuracy:27.1200%, loss:2.4147
For epoch 13 : test accuracy:30.4800%, loss:2.2911
For epoch 14 : test accuracy:32.9600%, loss:2.3621
For epoch 15 : test accuracy:36.0400%, loss:2.0152
For epoch 16 : test accuracy:38.5600%, loss:1.8636
For epoch 17 : test accuracy:40.2000%, loss:1.6837
For epoch 18 : test accuracy:42.6800%, loss:1.7554
For epoch 19 : test accuracy:44.8800%, loss:1.5541
